In [ ]:
%pip install -r '/home/pooya/w/EDS/requirements.txt'
%pip uninstall eds -y
%pip install '/home/pooya/w/EDS/diset/ds-0.1.tar.gz'
%pip list

# Load Modules

In [3]:
import sys
sys.path.insert(0,'..')

In [4]:
import json
import requests
import urllib
import warnings
import pandas as pd
import plotly.express as px
import eds

warnings.filterwarnings('ignore')

# Define Function to Get FIPS from (lat, lon)

In [5]:
def find_fips(lat, lon):
    
    # Encode Parameters 
    params = urllib.parse.urlencode(
        {
            'latitude': lat,
            'longitude':lon,
            'format':'json'
        }
    )

    # Get Response From API
    response = requests.get('https://geo.fcc.gov/api/census/block/find?' + params)

    # Parse json in Response
    data = response.json()

    return data['County']['FIPS']

# Calculate Disease Severity

In [6]:
result = eds.calculate_disease_severity(    
    data_path = "../notebook/data.csv",
    info_path = "../notebook/info.csv",
    number_of_repeat_year = 4,       
    number_applications_list = [0],
    genetic_mechanistic_list = ["Susceptible"],    
    parameters = eds.Parameters(),
    output_columns = ["SevMAX"],
    #TODO: Fix path_result
    path_result = "../notebook/result.csv",
)

result

,locationId,Date1,Date2,N_Days,latitude,longitude,number_applications,genetic_mechanistic,crop,SevMAX
0,ID_44.26405373_-92.9375238_2017_5/3/2017_129_Corn,20170503,20170909,129,44.264054,-92.937524,0,Susceptible,Corn,0.827648
1,ID_44.740789_-91.2186815_2017_5/3/2017_129_Corn,20170503,20170909,129,44.740789,-91.218682,0,Susceptible,Corn,0.818157
2,ID_44.97603894_-95.21102976_2017_5/3/2017_129_...,20170503,20170909,129,44.976039,-95.211030,0,Susceptible,Corn,0.813497
3,ID_45.1979665_-95.4234305_2017_5/3/2017_129_Corn,20170503,20170909,129,45.197966,-95.423430,0,Susceptible,Corn,0.829240
4,ID_45.83845837_-96.09193576_2017_5/3/2017_129_...,20170503,20170909,129,45.838458,-96.091936,0,Susceptible,Corn,0.828749
5,ID_45.9580605_-96.145168_2017_5/3/2017_129_Corn,20170503,20170909,129,45.958061,-96.145168,0,Susceptible,Corn,0.827834
6,ID_46.79832825_-97.46112752_2017_5/3/2017_129_...,20170503,20170909,129,46.798328,-97.461128,0,Susceptible,Corn,0.825143
7,ID_46.898326_-96.53989_2017_5/3/2017_129_Corn,20170503,20170909,129,46.898326,-96.539890,0,Susceptible,Corn,0.829477
8,ID_47.04440649_-97.04174625_2017_5/3/2017_129_...,20170503,20170909,129,47.044406,-97.041746,0,Susceptible,Corn,0.829151
9,ID_47.24408_-96.481627_2017_5/3/2017_129_Corn,20170503,20170909,129,47.244080,-96.481627,0,Susceptible,Corn,0.831311


# Add FIPS to Results

In [ ]:
#TODO: Find A Better Way To Do This
result["FIPS"] = result.apply(lambda x: find_fips(lat = x['latitude'], lon = x['longitude']), axis=1)

result

# Drop Duplicate FIPS & Select Maximum

In [ ]:
result = result.sort_values(["FIPS", "SevMAX"]).drop_duplicates(["FIPS"], keep='last').reset_index(drop=True)

# Load Geojson Counties

In [ ]:
with urllib.request.urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties = json.load(response)

# Plot SevMAX

In [ ]:
fig = px.choropleth_mapbox(
    result, 
    geojson=counties, 
    locations='FIPS', 
    color='SevMAX',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=3,
    center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.5,
)

fig.update_layout(
    height=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()